In [4]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

spark = SparkSession.builder.master("local[3]").appName("UdemyPyspark").getOrCreate()
print(spark)

#InferSchema as false, brings everything as String
df = (spark.read.format("csv").option("header","true")
    .option("inferSchema","true")
    .option("delimiter",";")
    .load("C:\\Users\\jrjos\\Documents\\Python Scripts\\PythonScripts\\Udemy-Pyspark\\Files\\covid_mock_file.csv"))
    

In [2]:
df.printSchema()
print("Dir command da Functions pyspark.sql para coluna regiao\n", dir(F.col("regiao")))
print("\nPrinting Spark DataFrame for 5 first records")
df.show(5)

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)

Dir command da Functions pyspark.sql para coluna regiao
 ['__add__', '__and__', '__bool__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__invert__', '__iter__', '__le__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__ror__', '__rpow__', '__rsub__', '__rtruediv__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__

# Seção com exemplos de SELECT e FILTER

In [3]:
df.select(F.col("regiao"), 
            F.col("estado"), 
            F.col("casosNovos"), 
            F.col("obitosNovos")).filter(F.col("regiao") == "Sul").show(10)

+------+------+----------+-----------+
|regiao|estado|casosNovos|obitosNovos|
+------+------+----------+-----------+
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
|   Sul|    PR|         0|          0|
+------+------+----------+-----------+
only showing top 10 rows



In [29]:
df.select(F.collect_set("estado")).show()

+--------------------+
| collect_set(estado)|
+--------------------+
|[AL, PA, CE, RN, ...|
+--------------------+



In [61]:
df.filter("obitosAcumulados>100").show()

+--------+------+----------+----------+---------------+-----------+----------------+
|  regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+--------+------+----------+----------+---------------+-----------+----------------+
|   Norte|    AM|15/04/2020|        70|           1554|         16|             106|
|   Norte|    AM|16/04/2020|       165|           1719|         18|             124|
|   Norte|    AM|17/04/2020|        90|           1809|         21|             145|
|   Norte|    AM|18/04/2020|        88|           1897|         16|             161|
|   Norte|    AM|19/04/2020|       147|           2044|         21|             182|
|   Norte|    AM|20/04/2020|       116|           2160|          3|             185|
|   Norte|    AM|21/04/2020|       110|           2270|          8|             193|
|   Norte|    AM|22/04/2020|       209|           2479|         14|             207|
|   Norte|    AM|23/04/2020|       409|           2888|         2

In [37]:
df.select(F.collect_set("estado").alias("Estados"), F.collect_set("regiao").alias("Regioes")).show()

+--------------------+--------------------+
|             Estados|             Regioes|
+--------------------+--------------------+
|[AL, PA, CE, RN, ...|[Nordeste, Sudest...|
+--------------------+--------------------+



In [39]:
df.filter(F.col("regiao") == 'Sudeste').select(F.collect_set("estado")).show(truncate=False)

+-------------------+
|collect_set(estado)|
+-------------------+
|[ES, MG, RJ, SP]   |
+-------------------+



In [ ]:
print("\nFiltering with & operator")
listFilter = df.filter(F.col("regiao") == 'Centro-Oeste').filter((F.col("estado") == 'DF')\
     | (F.col("estado") == 'MT')).where("casosNovos >0")

# Erro que eu não consegui identificar Pesquisar depois o porque salvar diretamente de Pyspark nao rolou
#listFilter.write.option("header", "true").format("csv").mode("append").save("ResultFilter")


In [ ]:
# Convertendo para Pandas e então salvando localmente em CSV funcionou
# preciso ver qual parametro precisa ser especificado para sobrescrever o arquivo, tive que mudar o nome do arquivo
# para poder salvar a segunda operação
pdlistFilter = listFilter.toPandas()
pdlistFilter.to_csv(r'C:\Users\JUNIOJX7\VsCode\Udemy-Pyspark\Files\output\datacsv\pdFilterTest2.csv', header=True, index=False)

In [ ]:
listFilter.select("regiao", "estado", "casosNovos").sortWithinPartitions("estado", ascending=True).show(5)

print("Collect_set com os Estados no DF")
listFilter.select(F.collect_set("estado")).show()

In [43]:
#Select usando Strings ao invés das Funcoes de Coluna (pyspark.functions) tem o mesmo efeito, porém através das Funcoes, é possível utilizar melhor os recursos
df.select("data", "regiao", "estado", "obitosAcumulados").\
    filter("regiao = 'Norte'").filter("casosNovos >50 and obitosAcumulados >10").show(truncate=False)

+----------+------+------+----------------+
|data      |regiao|estado|obitosAcumulados|
+----------+------+------+----------------+
|04/04/2020|Norte |AM    |12              |
|05/04/2020|Norte |AM    |14              |
|06/04/2020|Norte |AM    |19              |
|07/04/2020|Norte |AM    |23              |
|08/04/2020|Norte |AM    |30              |
|09/04/2020|Norte |AM    |40              |
|10/04/2020|Norte |AM    |50              |
|11/04/2020|Norte |AM    |53              |
|12/04/2020|Norte |AM    |62              |
|13/04/2020|Norte |AM    |71              |
|14/04/2020|Norte |AM    |90              |
|15/04/2020|Norte |AM    |106             |
|16/04/2020|Norte |AM    |124             |
|17/04/2020|Norte |AM    |145             |
|18/04/2020|Norte |AM    |161             |
|19/04/2020|Norte |AM    |182             |
|20/04/2020|Norte |AM    |185             |
|21/04/2020|Norte |AM    |193             |
|22/04/2020|Norte |AM    |207             |
|23/04/2020|Norte |AM    |234   

In [3]:
#Filter e Where tem o mesmo efeito, existe a possibilidade de aninhar Filter e Where como no exemplo do bloco de codigo seguinte
#Tambem no exemplo 2, é possivel escrever dentro do Filter o comando SQL
df.filter("regiao = 'Norte' and estado like 'R%'").show(5)
df.filter("regiao = 'Norte'").filter("estado not in('AM','AC','RO')").show(5)

listaregiao = ["Sudeste", "Sul", "Norte"]
#Uses the "~" is the NOT operator for "isin" to negate the value
df.filter(~F.col("regiao").isin(listaregiao)).show(5)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|30/01/2020|         0|              0|          0|               0|
| Norte|    RO|31/01/2020|         0|              0|          0|               0|
| Norte|    RO|01/02/2020|         0|              0|          0|               0|
| Norte|    RO|02/02/2020|         0|              0|          0|               0|
| Norte|    RO|03/02/2020|         0|              0|          0|               0|
+------+------+----------+----------+---------------+-----------+----------------+
only showing top 5 rows

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+---------

# Seção para exemplos de como usar a clausula WHERE

In [6]:
df.select(F.col("data"), 
    F.col("regiao"),
    F.col("estado"), 
    F.col("casosAcumulados"), 
    F.col("obitosAcumulados")).where(F.col("regiao") == 'Centro-Oeste').\
        where(F.col("estado") == 'GO').filter("casosNovos >0 and obitosAcumulados >0").orderBy(F.desc("casosAcumulados")).show(5)

+----------+------------+------+---------------+----------------+
|      data|      regiao|estado|casosAcumulados|obitosAcumulados|
+----------+------------+------+---------------+----------------+
|25/04/2020|Centro-Oeste|    GO|            506|              25|
|24/04/2020|Centro-Oeste|    GO|            486|              24|
|23/04/2020|Centro-Oeste|    GO|            453|              23|
|22/04/2020|Centro-Oeste|    GO|            438|              21|
|21/04/2020|Centro-Oeste|    GO|            421|              19|
+----------+------------+------+---------------+----------------+
only showing top 5 rows



# Seção para WithColum, Cast e "When"

In [ ]:
#WithColum adiciona uma nova coluna no DataFrame
new_df = df.withColumn("soma_casos_novos_acumulados", F.col("casosNovos")+F.col("casosAcumulados"))
new_df.printSchema()
new_df.where("casosNovos >0").show(5)

cast_df = new_df.select(F.col("soma_casos_novos_acumulados").cast("float"))
cast_df.printSchema()

In [ ]:
#Excelente exemplo usando de como aplicar WHEN (CASE WHEN) conditionals
df_status = df.withColumn("Status", F.when(F.col("casosNovos") >0, F.concat(df.casosNovos, F.lit(" CasosNovos"))).\
    otherwise("SemCasosNovos"))
df_status.where(df.casosNovos >0).show(5)

In [ ]:
df_DMY = df.withColumn("Status", F.when(F.col("casosNovos") >0, F.concat(df.casosNovos, F.lit(" CasosNovos"))).\
    otherwise("SemCasosNovos")).\
            withColumn("Dia", F.substring(df.data, 1,2)).\
            withColumn("Mes", F.substring(df.data, 4,2)).\
            withColumn("Ano", F.substring(df.data, 7,4))

#df_DMY.printSchema()

In [ ]:
df_DMY.select("data", "Dia", "Mes", "Ano").show(5)

In [ ]:
df_DMY.show(10);

In [ ]:
#Fazendo cast mantendo no mesmo df
df_DMY_casted = df.withColumn("Status", F.when(F.col("casosNovos") >0, F.concat(df.casosNovos, F.lit(" CasosNovos"))).\
    otherwise("SemCasosNovos")).\
            withColumn("Dia", F.substring(df.data, 1,2).cast("integer")).\
            withColumn("Mes", F.substring(df.data, 4,2).cast("integer")).\
            withColumn("Ano", F.substring(df.data, 7,4).cast("integer"))

df_DMY_casted.printSchema()

In [ ]:
df_DMY_casted.show(10)

# Struct Type e Header

In [3]:
schemaStruct = (StructType([
    StructField("BR_Regiao", StringType(), True),
    StructField("BR_UF", StringType(), True),
    StructField("DataRegistro", StringType(), True),
    StructField("Qtd_casosNovos", IntegerType(), True),
    StructField("Qtd_casosAcumulados", IntegerType(), True),
    StructField("Qtd_obitosNovos", IntegerType(), True),
    StructField("Qtd_obitosAcumulados", IntegerType(), True)
]))

dfStruct = (spark.read.format("csv").option("header","true")
    .option("inferSchema","false")
    .option("delimiter",";")
    .load("C:\\Users\\JUNIOJX7\\VsCode\\Udemy-Pyspark\\Files\\covid_mock_file.csv", schema=schemaStruct))

dfStruct.printSchema()

'''
Esse tipo de estrutura é muito valido quando se trabalha com arquivos sem o cabeçalho, ou se deseja alterar/padronizar
os nomes existentes nas colunas/cabeçalho de um determinado arquivo/tabela.
Atenção pois a quantidade de StructField tem que ser exatamente a mesma de colunas no arquivo, caso contrário nao tera nenhum erro,
mas a coluna sera desconsiderada.

Quando o header é marcado como False, e nenhum schema e passado como parametro no load, as colunas vem como 
_c0:
_c1:
_c...:

Outra maneira é usar o metodo toDF após o Load e especificar as colunas 
ex: load("C:\\Users\\JUNIOJX7\\Desktop\\old\\covid_mock_file.csv").toDF("A", "B", "C", ...)) 
'''


root
 |-- BR_Regiao: string (nullable = true)
 |-- BR_UF: string (nullable = true)
 |-- DataRegistro: string (nullable = true)
 |-- Qtd_casosNovos: integer (nullable = true)
 |-- Qtd_casosAcumulados: integer (nullable = true)
 |-- Qtd_obitosNovos: integer (nullable = true)
 |-- Qtd_obitosAcumulados: integer (nullable = true)



'\nEsse tipo de estrutura é muito valido quando se trabalha com arquivos sem o cabeçalho, ou se deseja alterar/padronizar\nos nomes existentes nas colunas/cabeçalho de um determinado arquivo/tabela.\nAtenção pois a quantidade de StructField tem que ser exatamente a mesma de colunas no arquivo, caso contrário nao tera nenhum erro,\nmas a coluna sera desconsiderada.\n\nQuando o header é marcado como False, e nenhum schema e passado como parametro no load, as colunas vem como \n_c0:\n_c1:\n_c...:\n\nOutra maneira é usar o metodo toDF após o Load e especificar as colunas \nex: load("C:\\Users\\JUNIOJX7\\Desktop\\old\\covid_mock_file.csv").toDF("A", "B", "C", ...)) \n'

In [5]:
dfStruct.filter("Qtd_casosNovos >10").filter("Qtd_obitosNovos >0").show(20)

+---------+-----+------------+--------------+-------------------+---------------+--------------------+
|BR_Regiao|BR_UF|DataRegistro|Qtd_casosNovos|Qtd_casosAcumulados|Qtd_obitosNovos|Qtd_obitosAcumulados|
+---------+-----+------------+--------------+-------------------+---------------+--------------------+
|    Norte|   RO|  19/04/2020|            18|                128|              1|                   4|
|    Norte|   RO|  22/04/2020|            24|                223|              1|                   5|
|    Norte|   RO|  25/04/2020|            38|                328|              2|                   7|
|    Norte|   AC|  13/04/2020|            13|                 90|              1|                   3|
|    Norte|   AC|  19/04/2020|            21|                163|              1|                   6|
|    Norte|   AC|  20/04/2020|            13|                176|              2|                   8|
|    Norte|   AC|  23/04/2020|            13|                227|        

# aula06_particionamento no Databricks

Consultar o arquivo salvo, diretamente no databricks community para execucao passo a passo, e depois repassar o codigo para execucao direta aqui conectando no DBcks, também fazer a Git push do diretorio de aulas

- Pesquisar as funções Partitionby, Repartition and Coalesce, mais exemplos e detalhes sobre RDD tbm

*https://sparkbyexamples.com/pyspark/pyspark-partitionby-example/*

*https://sparkbyexamples.com/spark/spark-partitioning-understanding/*

*https://spark.apache.org/docs/latest/rdd-programming-guide.html*

*https://sparkbyexamples.com/pyspark-rdd/*

## Particoes e RDD

In [7]:
from pyspark import SparkContext as SC


In [8]:
print(dfStruct.rdd.getNumPartitions())
dfStructPartition = dfStruct.repartition(F.col("BR_Regiao"))

rddStructPartition = dfStructPartition.rdd.repartition(3)
print(rddStructPartition.getNumPartitions())

1
3


[Row(BR_Regiao='Nordeste', BR_UF='MA', DataRegistro='19/02/2020', Qtd_casosNovos=0, Qtd_casosAcumulados=0, Qtd_obitosNovos=0, Qtd_obitosAcumulados=0),
 Row(BR_Regiao='Nordeste', BR_UF='MA', DataRegistro='20/02/2020', Qtd_casosNovos=0, Qtd_casosAcumulados=0, Qtd_obitosNovos=0, Qtd_obitosAcumulados=0),
 Row(BR_Regiao='Nordeste', BR_UF='MA', DataRegistro='21/02/2020', Qtd_casosNovos=0, Qtd_casosAcumulados=0, Qtd_obitosNovos=0, Qtd_obitosAcumulados=0),
 Row(BR_Regiao='Nordeste', BR_UF='MA', DataRegistro='22/02/2020', Qtd_casosNovos=0, Qtd_casosAcumulados=0, Qtd_obitosNovos=0, Qtd_obitosAcumulados=0)]

In [13]:
#Problema de referencia em configuracoes, que nao sabia aonde alterar, corrigido com copiando o executavel python.exe para python3.exe
#Estrutura do collect no RDD, significa [Particao][Linhas] aonde nas linhas é possivel fazer um slice n:n -> Particao 0 | linhas 0 : 2
rddStructPartition.glom().collect()[0][0:2]

[Row(BR_Regiao='Nordeste', BR_UF='MA', DataRegistro='19/02/2020', Qtd_casosNovos=0, Qtd_casosAcumulados=0, Qtd_obitosNovos=0, Qtd_obitosAcumulados=0),
 Row(BR_Regiao='Nordeste', BR_UF='MA', DataRegistro='20/02/2020', Qtd_casosNovos=0, Qtd_casosAcumulados=0, Qtd_obitosNovos=0, Qtd_obitosAcumulados=0)]

In [9]:
dfStructPartition.show(1)

+---------+-----+------------+--------------+-------------------+---------------+--------------------+
|BR_Regiao|BR_UF|DataRegistro|Qtd_casosNovos|Qtd_casosAcumulados|Qtd_obitosNovos|Qtd_obitosAcumulados|
+---------+-----+------------+--------------+-------------------+---------------+--------------------+
| Nordeste|   MA|  30/01/2020|             0|                  0|              0|                   0|
+---------+-----+------------+--------------+-------------------+---------------+--------------------+
only showing top 1 row



In [ ]:
dfStructPartition = dfStructPartition.withColumn("Dia", F.substring(dfStructPartition.DataRegistro, 1,2)).\
            withColumn("Mes", F.substring(dfStructPartition.DataRegistro, 4,2)).withColumn("Ano", F.substring(dfStructPartition.DataRegistro, 7,4))

In [ ]:
dfStructPartition.show(1)

In [6]:
# Usado para trazer todas as linhas do dataFrame por é possível utilizar estrutura de Matriz para recuperar os dados
# dfStructPartition.collect()[0] retornando o indice da linha e todas as colunas
# dfStructPartition.collect()[0][0] retornando o indice linha e coluna do conjunto 
# sendo possível utilizar o SLICE para colunas [N][n:n]
# Linha 0 Row(BR_Regiao='Nordeste', BR_UF='MA', DataRegistro='19/02/2020', Qtd_casosNovos=0, Qtd_casosAcumulados=0, Qtd_obitosNovos=0, Qtd_obitosAcumulados=0)
dfStructPartition.collect()[0][0:10]

NameError: name 'dfStructPartition' is not defined

In [ ]:
# Erro nao funciona para salvar localmente, mas funcionou para salvar no DataBricks Analisar depois, 
# pois mesmo direcionando o local para o mesmo caminho de load deu erro, mesmo com os parametros em options
# 16-JUN Deve ser algum erro de dependências entre as bibliotecas, como foi com o python3.exe, pois ao rodar o comando olhando para a file explorer ao lado, os arquivos começam a ser salvos
# porém acontece um erro de linkNative e a operação e encerrada, Estudar melhor: https://rollbar.com/blog/java-unsatisfiedlinkerror-runtime-error/#:~:text=lang.,a%20subclass%20of%20the%20java.

dfStructPartition.write.option("header", True).csv("C:/Users/JUNIOJX7/VsCode/Udemy-Pyspark/Files/output/datacsv2/FileOutput.csv")



## Teste para manual Struct

In [ ]:
dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)


In [ ]:
deptDF.show(truncate=False)

In [ ]:
deptDF.write.csv("C:/Users/JUNIOJX7/VsCode/Udemy-Pyspark/Files/output/datacsv2/FileOutput.csv")

# Aula 07 - Agrupamentos

Consultar codigo no Databricks community, pois o notebook estara com os comentarios lá, porém tentar aplicar essas funções com o conjunto de dados carregados localmente aqui também

In [ ]:
dfStructPartition.show(3)

In [ ]:
#SUM 
dfStructPartition.groupBy("BR_Regiao", "BR_UF").sum("Qtd_obitosAcumulados").show(5)

In [ ]:
#Count
dfStructPartition.groupBy("BR_Regiao").count().show()

In [ ]:
#min
dfStructPartition.groupBy("BR_Regiao").min("Qtd_obitosAcumulados").show()

In [ ]:
#MAX Simples
#dfStructPartition.groupBy("BR_Regiao").max("Qtd_obitosNovos").show()

#MAX com funcao de ordenacao sort()
# ERRO abaixo, procurar entender o porque usando ou não o agg e a funcao sort esta dando erro aqui
# O erro acontecia porque a descricao da coluna incorpora o nome da funcao, ficava max(Qtd_obitosNovos), o que gerava erro ao apontar somente para nome da coluna
# nesse caso deve-se colocar um alias, e utilizar a funcao agg, senao retorna erro.
#dfStructPartition.groupBy("BR_Regiao").agg(F.max("Qtd_obitosNovos")).sort(F.desc("Qtd_obitosNovos")).show()

dfStructPartition.groupBy("BR_Regiao").agg(F.max("Qtd_obitosNovos").alias("VolumeObitosTotal")).sort(F.desc("VolumeObitosTotal")).show()

In [ ]:
#AVG e MEAN --> Existe uma diferença na aplicação entre os métodos, nas aulas de DataScience com Python do CloudAcademy e no notebook relacionado tem mais detalhes entre as diferenças

dfStructPartition.groupBy("BR_Regiao").agg(F.mean("Qtd_obitosNovos").alias("MeanObitosNovos")).sort(F.desc("MeanObitosNovos")).show()
dfStructPartition.groupBy("BR_Regiao").agg(F.avg("Qtd_obitosNovos").alias("AvgObitosNovos")).sort(F.desc("AvgObitosNovos")).show()

In [ ]:
# Agrupando, aplicado funcoes de agregacao, e filtrando o resultado.
dfStructPartition.groupBy("BR_Regiao", "BR_UF").sum("Qtd_casosAcumulados", "Qtd_obitosAcumulados")\
    .filter(F.col("sum(Qtd_casosAcumulados)") > 5000).sort(F.desc("sum(Qtd_casosAcumulados)")).show()

In [ ]:
#Exemplo animal de agrupamento, por agregacao, utilizando outras funcoes de SQL atraves da Functions, e criando coluna para atribuir resultado no DF

df2_StructPartition = dfStructPartition.groupBy("BR_Regiao", "BR_UF").agg(F.sum("Qtd_casosNovos").alias("VolumeCasosNovos"),\
     F.sum("Qtd_obitosNovos").alias("VolumeObitosNovos"), F.format_number(F.avg("Qtd_casosNovos"), 2).alias("Avg CasosNovos"),\
     F.max("Qtd_obitosAcumulados").alias("Max_ObitosAcumulados")).\
         withColumn("pct_Casos", F.format_number((F.col("VolumeObitosNovos") / F.col("VolumeCasosNovos") * 100), 2).cast("float"))\
    .filter(F.col("VolumeCasosNovos") > 1000).sort(F.desc("VolumeCasosNovos"))

In [ ]:
df2_StructPartition.show()

# Aula 08 Funcoes de Agregacoes

In [ ]:
#Retornar a qtd de valores distintos de um dataframe

aproxFuncCount = dfStructPartition.select(F.approx_count_distinct("BR_UF")).collect()[0][0]
print(f"approx_count_distinct:  {aproxFuncCount}")

In [ ]:
#Retorna os valores do dataframe em uma lista, independente de valores repetidos ou não
dfStructPartition.select(F.collect_list("BR_UF")).show(truncate = True)

In [ ]:
#Retorna o valor do dataframe em uma lista, somente os valores unicos, nao retorna os duplicados
dfStructPartition.select(F.collect_set("BR_UF")).show(truncate = False)

In [ ]:
#Retorna o valor do dataframe em uma lista, nao retorna os valores unicos
dfStructPartition.select(F.collect_set("BR_Regiao")).show(truncate = False)

In [ ]:
#Retorna a qtd de itens unicos no dataframe
dfStructPartition.select(F.countDistinct("BR_UF")).show(truncate = False)

In [ ]:
#Retorna o valor do dataframe em uma lista, nao retorna os valores unicos
dfStructPartition.select(F.first("BR_UF")).show(truncate = False)

# Aula 09 Join

## Montagem dos DataFrames

In [ ]:
dataEmp = [
    (1, "Anderson", 10000),
    (2, "Kenedy", 20000),
    (3, "Billy", 23000),
    (4, "Andy", 23000),
    (5, "Mary", 24000),
    (6, "Eduardo", 19000),
    (7, "Mendes", 15000),
    (8, "Keyth", 18000),
    (9, "Truman", 21000),
]
schemaEmp = ["id", "name", "salary"]

dfEmp = spark.createDataFrame(data=dataEmp, schema=schemaEmp)
dfEmp.printSchema()
dfEmp.show(5)

dataPlace = [
    (1, "Delhi", "India"),
    (2, "Tamil nadu", "India"),
    (3, "London", "UK"),
    (4, "Sydney", "Australia"),
    (8, "New York", "USA"),
    (9, "California", "USA"),
    (10, "New Jersey", "USA"),
    (11, "Texas", "USA"),
    (12, "Chicago", "USA"),
]
schemaPlace = ["id", "place", "country"]

dfPlace = spark.createDataFrame(data=dataPlace, schema=schemaPlace)
dfPlace.printSchema()
dfPlace.show(5)


## Code's On

In [ ]:
#Basta usar o join, informar qual o dataframe, no on['coluna'], how='' é o tipo inner, outer, right ou left
# Pesquisar mais, porque as explicações da aula foram toscas
dfEmp.join(dfPlace, on=['id'], how='outer').show()

In [ ]:
dfEmp.join(dfPlace, on=['id'], how='left').show()

In [ ]:
dfEmp.join(dfPlace, on=['id'], how='right').show()

In [ ]:
dfEmp.join(dfPlace, on=['id'], how='inner').show()

In [ ]:
# left_anti pesquisar, mas basicamente traz todas as linhas do Dataframe a esquerda que nao tem na direita
# tem o anti que parece exercer a mesma funcao, pesquisar e anotar as diferenças depois
dfEmp.join(dfPlace, on=['id'], how='left_anti').show()

In [ ]:
# pesquisar mas basicamente traz todos os valores a esquerda que existem dados correspondentes a direita.
dfEmp.join(dfPlace, on=['id'], how='left_semi').show()

In [ ]:
# traz todos os registros
dfEmp.join(dfPlace, on=['id'], how='full').show()

# Aula 10 Funcoes de Window e Particoes

In [ ]:
from pyspark.sql.window import Window as W

## Montagem do DataFrame
Como no exemplo com o dataFrame de covid, tem muitos registros para acompanhar, vou criar o DataFrame proposto no curso com exemplo de um Dept de colaboradores e localidade para ver melhor os efeitos de cada uma das funcoes.

In [ ]:
data = [("Anderson","Sales","NY",90000),
    ("Kenedy","Sales","CA",86000),
    ("Kenny","Sales","CA",86000),
    ("Billy","Sales","NY",81000),
    ("Andy","Finance","CA",90000),
    ("Mary","Finance","NY",99000),
    ("Eduardo","Finance","NY",83000),
    ("Mendes","Finance","CA",79000),
    ("Keyth","Marketing","CA",80000),
    ("Truman","Marketing","NY",91000),
    ("Josa", "Finance", "AL",55000)
  ]
schema = ["name","dep_name","state","salary"]

dfAula10 = spark.createDataFrame(data=data, schema = schema)
dfAula10.printSchema()

In [ ]:
dfAula10.show()

## Code's On

In [ ]:
#Cria resultado da funcao de Window pela particao do Departamento, ordenada pelo Salario decrescente dos colaboradores
wt01 = W.partitionBy(F.col("dep_name")).orderBy(F.desc("salary"))

In [ ]:
#RowNumber, lista o numero das linhas por Departamento do maior salario para o menor
dfAula10.withColumn("rowNumber", F.row_number().over(wt01)).show()

In [ ]:
# Fiz o teste colocando uma nova Window WindowTest01_1 para ver o resultado de ter o departamento e o estado particionados
# E nesse caso, o resultado mostra o meu salario primeiro, mesmo sendo o menor de todos, pois a particao estado AL tem prioridade ordem alfabetica
# que os demais, no exemplo acima, como o Estado não é uma partição, o meu nome aparece em último entre os departamentos.
wt01_1 = W.partitionBy(F.col("dep_name"), F.col("state")).orderBy(F.desc("salary"))

In [ ]:
dfAula10.withColumn("rowNumber", F.row_number().over(wt01_1)).show()

In [ ]:
# Rank mostra a posicao dos valores Pelo Particionamento, ordenado pelo Salario, considerando ordem alfabetica para os demais itens
# por isso Kenedy esta na rank 2 acima do Kenny também com rank 2 com base no Dept e Salario, e deixa a lacuna para o rank 3 
# (pertencente a Kenny segundo em desempate no rank 2) colocando Billy com o Rank 4
# Resumindo a Rank coloca em Rank os itens estipulados pelo Particionamento e Orderby, agrupando todos os valores empatados, porem guardando a posicao 1,2,[2,2,2],6,7
# Por isso em Sales vemos 1,2,2,4 ele mantém a posição do elemento com Rank
dfAula10.withColumn("Rank", F.rank().over(wt01)).show()

In [ ]:
# dense_rank diferente do Rank, preenche a posição seguinte a que teve agrupamento por empate do Rank anterior, por isso aqui Billy esta com Rank 3, ao invés de 4
dfAula10.withColumn("Dense_Rank", F.dense_rank().over(wt01)).show()

In [ ]:
# Pesquisar, explicacao tosca
dfAula10.withColumn("Percent_Rank", F.percent_rank().over(wt01)).show()

In [ ]:
# ntile(nSlices) por nSlices ele agrupa os valores do particionamento nessa qtd atribuido na função
# agrupando um subconjunto de dados ordenados
# com isso poderiamos dizer que o Grupo 1 contem o maiores salarios por departamento, grupo 2 os segundos, e grupo 3 os que ganham menos
# Se tivessemos mais pessoas ele agruparia 3 pessoas por Grupo ->> TO-DO colocar mais pessoas por departamento e rodar novamente pra ver a efetividade
dfAula10.withColumn("Ntile", F.ntile(3).over(wt01)).show()

In [ ]:
# **Pesquisar mais**, pega a distribuicao culmulativa da particao
dfAula10.withColumn("cume_dist", F.cume_dist().over(wt01)).show()

In [ ]:
# Explicacao Tosca, mas funcao pode ser importante, pesquisar mais a respeito pode ser valido PRA CARALHO pra DataScience
# Mas basicamente, escolhe qual a coluna que quer atribuir o Lag, e a Qtd de Lag, que signfica voltar essa Qtd na Coluna dentro do DataFrame
# Por isso Eduardo tem o Lag de 99000, Josa de 83000
dfAula10.withColumn("Lag", F.lag("salary", 2).over(wt01)).show()

In [ ]:
# Similar ao Lag, porém ao invés de percorrer para Tras, vai para Frente no Conjunto.
dfAula10.withColumn("Lead", F.lead("salary", 2).over(wt01)).show()

# Datatypes Complexos

In [11]:
from pyspark.sql.types import ArrayType, DoubleType, IntegerType, LongType, StringType, StructType, StructField, BooleanType, MapType
from pyspark.sql import Row

In [12]:
'''
Trabalhando com Arrays, no import acima, trouxe alguns tipos do sql Types, o legal aqui foi trabalhar com os Arrays como pontos
No Bloco abaixo, a definição do schema é interessante mostrar que o campo Points e o user_state tem tipos diferentes que eu havia visto ate aqui
O legal é como os valores são mostrados em operacoes no DF, Ver a execucao do bloco 3 df.show()
'''
data = [Row("Kenny","",10,[50,90,80],{"status":"Active"}), 
        Row("Elis","Robert",20,[10,56,43,20],{"status":"Inactive"}), 
        Row("Myck","Mendes",30,[18,50,32],{"status":"Active"}), 
        Row("Edson","Eliot",40,[60,87,3],{"status":"Active"}) 
      ]
rdd = spark.sparkContext.parallelize(data)

In [13]:
scheme = StructType([
         StructField('firstname', StringType(), True),
         StructField('middlename', StringType(), True),
         StructField('age', IntegerType(), True),
         StructField("points", ArrayType(StringType()), True),
         StructField("user_state", MapType(StringType(),StringType()), True)        
         ])

In [14]:
df = rdd.toDF(schema=scheme)
display(df)

DataFrame[firstname: string, middlename: string, age: int, points: array<string>, user_state: map<string,string>]

In [15]:
'''
Da pra ver que os pontos são mostrados exatamente como uma Lista (array), e o ponto de Map, é atribuido como chave:valor
'''
df.show()

+---------+----------+---+----------------+--------------------+
|firstname|middlename|age|          points|          user_state|
+---------+----------+---+----------------+--------------------+
|    Kenny|          | 10|    [50, 90, 80]|  {status -> Active}|
|     Elis|    Robert| 20|[10, 56, 43, 20]|{status -> Inactive}|
|     Myck|    Mendes| 30|    [18, 50, 32]|  {status -> Active}|
|    Edson|     Eliot| 40|     [60, 87, 3]|  {status -> Active}|
+---------+----------+---+----------------+--------------------+



In [16]:
#Apois usar a funcao Explode, os pontos sao atribuidos a cada linha no DF
df02 = (df.withColumn("point", F.explode("points"))
  .select("firstname","point","user_state.status")        
)

display(df02)

DataFrame[firstname: string, point: string, status: string]

In [17]:
df02.show()

+---------+-----+--------+
|firstname|point|  status|
+---------+-----+--------+
|    Kenny|   50|  Active|
|    Kenny|   90|  Active|
|    Kenny|   80|  Active|
|     Elis|   10|Inactive|
|     Elis|   56|Inactive|
|     Elis|   43|Inactive|
|     Elis|   20|Inactive|
|     Myck|   18|  Active|
|     Myck|   50|  Active|
|     Myck|   32|  Active|
|    Edson|   60|  Active|
|    Edson|   87|  Active|
|    Edson|    3|  Active|
+---------+-----+--------+



In [ ]:
#Aqui o filter, usa a funcao array_contains, para filtrar a coluna do tipo array, e qual o valor que deseja buscar, e cria (withColum) a coluna get_point, e busca somente o elemento 2 da coluna points.
#isso com o uso da funcao element_at(Coluna, index), resultado no bloco abaixo, pode ser comparado com os pontos do resultado do bloco acima.
df03 = df.filter(F.array_contains(F.col("points"), "50")).withColumn("get_point", F.element_at(F.col("points"), 2))
display(df03)

In [ ]:
df03.show()

In [ ]:
df04 = df.withColumn("get_point", F.element_at(F.col("points"), 1))
display(df04)

In [ ]:
df04.show()

In [ ]:
#Agrupamento agreegado do DF acima, pelo Status dos usuários.
df05 = df04.groupBy("firstname", "user_state.status").agg(F.collect_set("get_point").alias("Points"))

display(df05)

In [ ]:
df05.show()

In [ ]:
df05 = df05.withColumnRenamed("Points","New_Points")
df05.show(truncate=False)

# User Defined Functions, Json, Parquet

foi pessima, buscar mais referencias por fora porque no curso foi ridiculamente péssimo

# Spark SQL

In [ ]:
df.show()

In [3]:
from pyspark.sql.functions import input_file_name

#Cria uma VIEW temporaria, simples, segue a estrututa, especifica o tipo de arquivo e o local, e por boa pratica, incluir o filename, pois eh possivel, ler uma lista de arquivos.

(spark.read.option("header","true")
    .option("inferSchema","true")
    .option("delimiter",";")
    .csv("C:\\Users\\JUNIOJX7\\VsCode\\Udemy-Pyspark\\Files\\covid_mock_file.csv")
    .withColumn("fileName", input_file_name())).createOrReplaceTempView("vw_tbl_covid")

In [4]:
#Em seguida é possivel executar qualquer comando SQL, utilizando a funcao sql.
spark.sql("SELECT * FROM vw_tbl_covid WHERE regiao='Norte' AND (estado='AM' OR estado='RR') AND obitosAcumulados >0 AND data BETWEEN '01/03/2020' AND '30/03/2020';").show(10)

+------+------+----------+----------+---------------+-----------+----------------+--------------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|            fileName|
+------+------+----------+----------+---------------+-----------+----------------+--------------------+
| Norte|    AM|25/03/2020|         7|             54|          1|               1|file:///C:/Users/...|
| Norte|    AM|26/03/2020|        13|             67|          0|               1|file:///C:/Users/...|
| Norte|    AM|27/03/2020|        14|             81|          0|               1|file:///C:/Users/...|
| Norte|    AM|28/03/2020|        30|            111|          0|               1|file:///C:/Users/...|
| Norte|    AM|29/03/2020|        29|            140|          0|               1|file:///C:/Users/...|
| Norte|    AM|30/03/2020|        11|            151|          0|               1|file:///C:/Users/...|
| Norte|    AM|01/04/2020|        25|            200|          0

In [5]:
spark.sql("SELECT regiao, COUNT(DISTINCT(estado)) as Estados FROM vw_tbl_covid GROUP BY regiao HAVING Estados >2;").show()
#spark.sql("SELECT * FROM vw_tbl_covid;").show(5)

+------------+-------+
|      regiao|Estados|
+------------+-------+
|    Nordeste|      9|
|         Sul|      3|
|     Sudeste|      4|
|Centro-Oeste|      4|
|       Norte|      7|
+------------+-------+



In [9]:
spark.sql("SELECT DISTINCT estado, regiao FROM vw_tbl_covid WHERE regiao = 'Norte'").show()

+------+------+
|estado|regiao|
+------+------+
|    PA| Norte|
|    RR| Norte|
|    AC| Norte|
|    AP| Norte|
|    RO| Norte|
|    AM| Norte|
|    TO| Norte|
+------+------+



In [ ]:
spark.sql("SELECT SUM(qtd_estados) as Total_UF FROM (SELECT regiao, COUNT(DISTINCT(estado)) as qtd_estados FROM vw_tbl_covid GROUP BY 1);").show()

In [ ]:

spark.sql("SELECT regiao, SUM(UFs) AS TOTAL_UF FROM (SELECT regiao, COUNT(DISTINCT(estado)) as UFs FROM vw_tbl_covid GROUP BY regiao) GROUP BY regiao ORDER BY regiao;").show()


In [ ]:
#Conceito captado, interessante eh que dinamicamente em tempo de execucao e possivel estabelecer criterios e salvar datasets, views e mesmo tabelas
 spark.sql("SELECT * FROM vw_tbl_covid WHERE data <='30/01/2020' AND regiao='Sul' AND obitosAcumulados <>0\
    UNION ALL SELECT * FROM vw_tbl_covid WHERE data <='30/01/2020' AND regiao='Sudeste' AND obitosAcumulados <>0;").createOrReplaceTempView("vw_union_sulsudeste")
 

In [ ]:
spark.sql("SELECT DISTINCT data FROM vw_tbl_covid WHERE casosNovos <>0;").show()

In [ ]:
spark.sql("SELECT DISTINCT SUBSTRING(data, 4,2) FROM vw_tbl_covid;").show()

In [ ]:
#FUNCOES DE JANELA NO SQL
#CRIANDO O DF

data = [("Anderson","Sales","NY",90000),
    ("Kenedy","Sales","CA",86000),
    ("Kenny","Sales","CA",86000),
    ("Billy","Sales","NY",81000),
    ("Andy","Finance","CA",90000),
    ("Mary","Finance","NY",99000),
    ("Eduardo","Finance","NY",83000),
    ("Mendes","Finance","CA",79000),
    ("Keyth","Marketing","CA",80000),
    ("Truman","Marketing","NY",91000),
    ("Josa", "Finance", "AL",55000)
  ]
schema = ["name","dep_name","state","salary"]

spark.createDataFrame(data=data, schema=schema).createOrReplaceTempView("vw_tbl_emp_window_f")

In [ ]:
#Usando o Rank
spark.sql("SELECT name, dep_name, RANK() OVER(PARTITION BY dep_name ORDER BY salary DESC) AS rank FROM vw_tbl_emp_window_f;").show()

In [ ]:
#Usando Dense_rank
spark.sql("SELECT name, dep_name, state, DENSE_RANK() OVER(PARTITION BY dep_name ORDER BY salary DESC) AS rank FROM vw_tbl_emp_window_f;").show()

In [ ]:
#Usando Dense_rank com os parametros ROWS BETWEEN....
spark.sql("SELECT name, dep_name, DENSE_RANK() OVER(PARTITION BY dep_name ORDER BY salary DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rank FROM vw_tbl_emp_window_f;").show()

In [ ]:
spark.sql("SELECT name, salary, dep_name, ROW_NUMBER() OVER(PARTITION BY dep_name ORDER BY salary DESC) AS rank FROM vw_tbl_emp_window_f;").show()